In [1]:
--- Libraries:
arguments = require 'Settings.arguments'
constants = require 'Settings.constants'
card_to_string = require 'Game.card_to_string_conversion'
card_tools = require 'Game.card_tools'
game_settings = require 'Settings.game_settings'
Plot = require 'itorch.Plot'
nninit = require 'nninit'
require 'torch'
require 'math'
require 'Tree.tree_builder'
require 'Tree.tree_data_generation'
require 'Tree.tree_visualiser'
require 'nn'
require 'Tree.tree_cfr'
require 'nngraph'
require 'optim'
require 'image'
require 'NN.strategy_net_builder'
require 'NN.strategy_net_trainer'



In [2]:
--- Create the tree
builder = PokerTreeBuilder()
--- Parameters for the tree
params = {}
params.root_node = {}
params.root_node.board = card_to_string:string_to_board('')
params.root_node.street = 1
params.root_node.current_player = constants.players.P1
params.root_node.bets = arguments.Tensor{200, 200}
--- CFR Solver
starting_ranges = arguments.Tensor(constants.players_count, game_settings.card_count)
starting_ranges[1]:copy(card_tools:get_uniform_range(params.root_node.board))
starting_ranges[2]:copy(card_tools:get_uniform_range(params.root_node.board))

In [3]:
--- BUild trees
tree2000 = builder:build_tree(params)
tree800 = builder:build_tree(params)
treeNoWarm = builder:build_tree(params)
treeNNWarm = builder:build_tree(params)
treeTargWarm = builder:build_tree(params)

--- build data
---game = TreeData(tree)
---game0 = TreeData(tree0)
---game1 = TreeData(tree1)
---game:get_training_set(tree,1)

In [4]:
--- Building CFR Solver
tree800_cfr = TreeCFR()
print("Solver")
tree800_cfr:run_cfr(tree800, starting_ranges,800)

Solver


In [5]:
--- Get Data from 2000 iterations
tree_data800 = TreeData(tree800)
tree_data800:get_training_set(tree800,1)

### Training the Model

In [6]:
-- Building the neural net model
strategy_nn = StrategyNN()
nn_model = strategy_nn.model
-- Building trainer:
nn_trainer = NNTrainer(tree_data800,nn_model)
-- Options:
opt = {}
opt.learningRate = 0.001
opt.momentum = 0.95
opt.validate = false

In [7]:
current_criterion = nn.SmoothL1Criterion()
train_loss,test_loss = nn_trainer:train(nn_model,current_criterion,opt,10000)

In [8]:
pred_strat = nn_model:forward({nn_trainer.all_data.features,nn_trainer.all_data.masks})
true_strat =nn_trainer.all_data.targets
print(torch.mean(torch.abs(pred_strat- true_strat)))

0.23437187720974


### No Warm Start

In [9]:
--- Building CFR Solver
treeNoWarm_cfr = TreeCFR()
print("Solver")
treeNoWarm_cfr:run_cfr(treeNoWarm, starting_ranges,800)

Solver


### Neural Net Start 

In [10]:
--- NNWarm Starting
dataNN = TreeData(treeNNWarm)
dataNN:get_training_set(treeNNWarm,1)
dataNN:generate_new_initial_strategies(treeNNWarm,nn_trainer)

In [ ]:
--- Building CFR Solver
treeNNWarm_cfr = TreeCFR()
treeNNWarm_cfr:run_cfr(treeNNWarm, starting_ranges,800)

### Target Warm Start 

In [ ]:
--- Target Warm Starting
dataTarg = TreeData(treeTargWarm)
dataTarg:get_training_set(treeTargWarm,1)
dataTarg:warm_start_from_targets(treeTargWarm,tree800)

In [ ]:
--- Building CFR Solver
treeTargWarm_cfr = TreeCFR()
treeTargWarm_cfr:run_cfr(treeTargWarm, starting_ranges,800)

### 2000 Iterations

In [ ]:
--- Building CFR Solver
tree2000_cfr = TreeCFR()
print("Solver")
tree2000_cfr:run_cfr(tree2000, starting_ranges,2000)

In [ ]:
--- Comparing with 2000
function compare_cfrs(tree_cfr1,tree_cfr2000)
    local difference = {}
    local table1 = tree_cfr1.total_strategies
    local table2 = tree_cfr2000.total_strategies
    
    for i = 1, #table1 do
        difference[i] = torch.sqrt(torch.sum(torch.pow((table1[i] - table2[#table2]),2)))
    end
    return torch.FloatTensor(difference)
end
--- Comparing 2000 with itself
function get_convergence(tree_cfr)
    local difference = {}
    local table_cfr = tree_cfr.total_strategies
    local end_strategy = table_cfr[#table_cfr]
    
    for i = 1, #table_cfr do
        difference[i] = torch.sqrt(torch.sum(torch.pow((table_cfr[i] - end_strategy),2)))
    end
    return torch.FloatTensor(difference)
end

In [ ]:
diff_NoW = compare_cfrs(treeNoWarm_cfr,tree2000_cfr)---:narrow(1,101,700)
diff_NNW = compare_cfrs(treeNNWarm_cfr,tree2000_cfr)---:narrow(1,101,700)
diff_TargW = compare_cfrs(treeTargWarm_cfr,tree2000_cfr)---:narrow(1,101,700)

In [ ]:
local num_iter = torch.range(1,diff_NoW:size(1))
local plot = Plot()

plot:line(num_iter,diff_NoW,'red' ,'NW: Strategy without WS')
plot:line(num_iter,diff_NNW,'blue' ,'NN : Strategy initialized with NN')
plot:line(num_iter,diff_TargW,'green' ,'SNN : Strategy initialized with target values')
plot:legend(true):title('MSE between strategies and 2000 iter CFR ')
plot:draw()

In [ ]:
convergence = get_convergence(tree2000_cfr):narrow(1,101,700)


In [ ]:
local num_iter = torch.range(1,convergence:size(1))
local plot = Plot()

plot:line(num_iter,convergence,'red' ,'MSE: 2000 iterations')
plot:line(num_iter,diff_NoW:narrow(1,101,700),'blue' ,'NW: Strategy without WS')
plot:line(num_iter,diff_NNW:narrow(1,101,700),'green' ,'NN : Strategy initialized with NN')
plot:legend(true):title('2000 iterations run vs its Final Policy')
plot:draw()

### Second 2000 iterations

In [ ]:
--- Building CFR Solver
tree2001 = builder:build_tree(params)
tree2001_cfr = TreeCFR()
print("Solver")
tree2001_cfr:run_cfr(tree2001, starting_ranges,2000)

In [ ]:
diff_2000 = compare_cfrs(tree2000_cfr,tree2001_cfr)---:narrow(1,10,700)

In [ ]:
local num_iter = torch.range(1,diff_2000:size(1))
local plot = Plot()

plot:line(num_iter,diff_2000,'red' ,'NW: Strategy without WS')
plot:legend(true):title('MSE between strategies and 2000 iter CFR ')
plot:draw()

In [ ]:
--- Comparing with 2000
function compare2000(tree_cfr1,tree_cfr2000)
    local difference = {}
    local table1 = tree_cfr1.total_strategies
    local table2 = tree_cfr2000.total_strategies
    
    for i = 1, #table1 do
        difference[i] = torch.sqrt(torch.sum(torch.pow((table1[i] - table2[#table1]),2)))
    end
    return torch.FloatTensor(difference)
end

In [ ]:
diff2000 = get_convergence(tree2000_cfr):narrow(1,101,1899)
diff2001 = get_convergence(tree2001_cfr):narrow(1,101,1899)

In [ ]:
local num_iter = torch.range(1,diff2000:size(1))
local plot = Plot()

plot:line(num_iter,diff2000,'red' ,'NW: Strategy without WS')
plot:line(num_iter,diff2001,'blue' ,'NW: Strategy without WS')
plot:legend(true):title('MSE between strategies and 2000 iter CFR ')
plot:draw()